In [3]:
import utils
import numpy as np
import pandas as pd

In [4]:
INPUT_PATH = "test_results/test_neurons_results.csv"

neuron_results_df = pd.read_csv(INPUT_PATH)
neuron_results_df

,Unnamed: 0,layer,neuron,explainer,NeuronViewer,Original,Summary,Highlight,HighlightSummary,AVHS
0,0,0,286,Meta-Llama-3.1-8B-Instruct,words related to comparison.,comparisons and relative differences.,words that indicate a comparison or a contrast.,"words that can be paired with ""compar"".",words that indicate similarity or contrast.,words that mean 'different' or 'similar' when ...
1,1,10,1838,Meta-Llama-3.1-8B-Instruct,phrases describing positions or situations in...,individuals or groups who find themselves in s...,words or phrases that convey a sense of someth...,third person singular present tense verb forms...,first/second/third person reflexive forms of t...,"the token ""in"" together with other words that ..."
2,2,20,193,Meta-Llama-3.1-8B-Instruct,verbs indicating questioning or challenging be...,phrases related to social criticism and accoun...,"phrases related to criticism, condemnation, or...",words related to social hierarchy and inequali...,"words related to criticism, confrontation, and...",words related to negative actions or behaviors...
3,3,30,1685,Meta-Llama-3.1-8B-Instruct,expressions of gratitude and agreeing to rece...,opt-in forms for email newsletters and such re...,"phrases related to subscriptions, permission t...","text segments related to newsletters, subscrip...","phrases related to user agreements, permission...",phrases related to taking or being a step.
4,4,40,431,Meta-Llama-3.1-8B-Instruct,"numbers related to time, dates, and measureme...","data about upcoming events, specifically sport...","numbers, especially times and dates in the for...",time-related expressions and numerical values.,colons and numerals.,"time-related information, specifically items r..."


In [5]:
def get_embedding(text_to_embed):
	# Embed a line of text
	response = utils.embedding_with_backoff(
    	model= "Meta-Llama-3.1-8B-Instruct",
    	input=text_to_embed
	)
	# Extract the AI output embedding as a list of floats
	embedding = response["data"][0]["embedding"]

	return embedding

def cosSim(A, B):
  A=np.array(A).flatten()
  B=np.array(B).flatten()
  return np.dot(A,B)/(np.linalg.norm(A)*np.linalg.norm(B))

In [6]:
modes = ["Original", "Summary", "Highlight", "HighlightSummary", "AVHS"]
AdaCS_results = {"layer":[], "neuron":[]}
for mode in modes:
    AdaCS_results[mode] = []

for i, row in neuron_results_df.iterrows():
    layer = row["layer"]
    neuron = row["neuron"]
    NV_Explan = row["NeuronViewer"]
    AdaCS_results["layer"].append(layer)
    AdaCS_results["neuron"].append(neuron)
    
    for mode in modes:
        prompt_Explan_Vector = get_embedding(row[mode])
        NV_Explan_Vector = get_embedding(NV_Explan)
        AdaCS_results[mode].append(cosSim(prompt_Explan_Vector, NV_Explan_Vector))

new_df = pd.DataFrame(AdaCS_results)
new_df

AuthenticationError: No API key provided. You can set your API key in code using 'openai.api_key = <API-KEY>', or you can set the environment variable OPENAI_API_KEY=<API-KEY>). If your API key is stored in a file, you can point the openai module at it with 'openai.api_key_path = <PATH>'. You can generate API keys in the OpenAI web interface. See https://platform.openai.com/account/api-keys for details.

In [5]:
new_df.to_csv(INPUT_PATH.split(".")[0] + "_adaCS.csv")

In [6]:
for mode in modes:
    avg = np.mean(new_df[mode])
    print(mode, avg)

Original 0.8443929253179865
Summary 0.8692494952946233
Highlight 0.8712292357118165
HighlightSummary 0.8810450174990387
AVHS 0.8456201711838094
